# To do list
- switch to storage
- ingest csvs like storage class
- figure out traversals

In [10]:
import knowledge_graph as g
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
from storage import Metadata, Storage

In [11]:
load_dotenv(find_dotenv())

True

In [24]:
graph = g.Graph(os.getenv('GRAPH_URI'), os.getenv('GRAPH_USER') , os.getenv('GRAPH_PASS'))


In [13]:
csv_files = os.listdir("../../dev/data/Football Example/")

In [18]:
csv_files[0]

'appearances.csv'

In [22]:
# m = metadata("../../dev/data/Football Example/" + csv_files[0])
# "../../dev/data/Football Example/apperances.csv"

FileNotFoundError: [Errno 2] No such file or directory: '../../dev/data/Football Example/appearances.desc'

# Pull all test CSVs
- pull all test dfs into a dict/hash map, pull their columns
- pull their PK/FK pairs
  - all PKs are first column
  - check to see if FK appears in column list
  - write another check to find all PK/FKs in dataframes based on keywords ('ID', 'KEY')
    - check across all dataframes for matching keys

- example
  - dataframe w/ GameID
  - dataframe w/ PlayerID
  - dataframe w/ GameID, PlayerID, etc
    - this would be composite key if multiple IDs
  - dataframe w/ GameID (FK), PlayerID (FK), EventID (PK)
    - has one PK, how would we identify?
    - would it be same as node name + id/key?
- non-automated example
  - make a file that makes PK/FK pairs w/ table names
    - relationships.txt
    - relationships.json
- 

In [38]:
df_dict={}
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv("../../dev/data/Football Example/" + f, encoding='latin-1')
    df_name = f.split("\\")[-1].split('.csv')[0]
    df_keys = []
    for column in df.columns.tolist():
        if ("id") in column.lower():
            df_keys.append(column)
            
    df_dict[df_name] = {"columns" : df.columns.tolist(),
                        "keys": df_keys}
                        
      
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1].split('.csv')[0])
      
    # print the content
    # print('Content:')
    # display(df)
    # print()

Location: appearances.csv
File Name: appearances
Location: games.csv
File Name: games
Location: leagues.csv
File Name: leagues
Location: players.csv
File Name: players
Location: shots.csv
File Name: shots
Location: teams.csv
File Name: teams
Location: teamstats.csv
File Name: teamstats


# Turn Data Frames into Nodes
- assuming after turning all file types/databases into pandas dataframes we can turn them into nodes
  - also assuming we might not even have to pull full DFs, just metadata, columns, etc.

In [43]:
df_dict.items()

dict_items([('appearances', {'columns': ['gameID', 'playerID', 'goals', 'ownGoals', 'shots', 'xGoals', 'xGoalsChain', 'xGoalsBuildup', 'assists', 'keyPasses', 'xAssists', 'position', 'positionOrder', 'yellowCard', 'redCard', 'time', 'substituteIn', 'substituteOut', 'leagueID'], 'keys': ['gameID', 'playerID', 'leagueID']}), ('games', {'columns': ['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID', 'homeGoals', 'awayGoals', 'homeProbability', 'drawProbability', 'awayProbability', 'homeGoalsHalfTime', 'awayGoalsHalfTime', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA'], 'keys': ['gameID', 'leagueID', 'homeTeamID', 'awayTeamID']}), ('leagues', {'columns': ['leagueID', 'name', 'understatNotation'], 'keys': ['leagueID']}), ('players', {'columns': ['playerID', 'name'], 'keys': ['playerID']}), ('shots', {'columns': ['gameID', 'shooterID', 'assisterID', 'minute', 'situatio

In [51]:
for name, dict in df_dict.items():
    print(name, dict.get("keys"))
    keys = dict.get("keys")
    print(name, dict.get("columns"))
    columns = dict.get("columns")
    print("Creating node!!!")
    graph.add_table_node(name, dict.get("keys"), dict.get("columns"), "TABLE")

appearances ['gameID', 'playerID', 'leagueID']
appearances ['gameID', 'playerID', 'goals', 'ownGoals', 'shots', 'xGoals', 'xGoalsChain', 'xGoalsBuildup', 'assists', 'keyPasses', 'xAssists', 'position', 'positionOrder', 'yellowCard', 'redCard', 'time', 'substituteIn', 'substituteOut', 'leagueID']
Creating node!!!
games ['gameID', 'leagueID', 'homeTeamID', 'awayTeamID']
games ['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID', 'homeGoals', 'awayGoals', 'homeProbability', 'drawProbability', 'awayProbability', 'homeGoalsHalfTime', 'awayGoalsHalfTime', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA']
Creating node!!!
leagues ['leagueID']
leagues ['leagueID', 'name', 'understatNotation']
Creating node!!!
players ['playerID']
players ['playerID', 'name']
Creating node!!!
shots ['gameID', 'shooterID', 'assisterID']
shots ['gameID', 'shooterID', 'assisterID', 'minute', 'si

In [ ]:
for table1, keys1 in table_primary_keys.items():
    for table2, keys2 in table_primary_keys.items():
        # Avoid comparing a table to itself
        if table1 != table2:
            # Check if there are shared primary keys
            shared_keys = set(keys1) & set(keys2)
            if shared_keys:
                # Append the relationship to the list
                table_relationships.append((table1, table2, shared_keys))

In [70]:
table_relationships = []
for table1, dict1 in df_dict.items():
    keys1 = dict1.get("keys")
    for table2, dict2 in df_dict.items():
        keys2 = dict2.get("keys")
        if table1 != table2:
            shared_keys = set(keys1) & set(keys2)
            if shared_keys:
                for shared_key in shared_keys:
                    table_relationships.append((table1, table2, shared_keys))

In [77]:
table_relationships[0][2]

{'gameID', 'leagueID'}

In [71]:
table_relationships

[('appearances', 'games', {'gameID', 'leagueID'}),
 ('appearances', 'leagues', {'leagueID'}),
 ('appearances', 'players', {'playerID'}),
 ('appearances', 'shots', {'gameID'}),
 ('appearances', 'teamstats', {'gameID'}),
 ('games', 'appearances', {'gameID', 'leagueID'}),
 ('games', 'leagues', {'leagueID'}),
 ('games', 'shots', {'gameID'}),
 ('games', 'teamstats', {'gameID'}),
 ('leagues', 'appearances', {'leagueID'}),
 ('leagues', 'games', {'leagueID'}),
 ('players', 'appearances', {'playerID'}),
 ('shots', 'appearances', {'gameID'}),
 ('shots', 'games', {'gameID'}),
 ('shots', 'teamstats', {'gameID'}),
 ('teams', 'teamstats', {'teamID'}),
 ('teamstats', 'appearances', {'gameID'}),
 ('teamstats', 'games', {'gameID'}),
 ('teamstats', 'shots', {'gameID'}),
 ('teamstats', 'teams', {'teamID'})]